In [2]:
from openmm import unit
from openmmtools import testsystems, states, mcmc
from openmmtools.multistate import ParallelTemperingSampler, MultiStateReporter
import tempfile
import os

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [16]:
from openmm.app import *

pdb = PDBFile('/home/jakub/phd/openmm-md/data/241010_FoldingUponBinding/output/SUMO-1C/241128-MetaD/sumo1c_equilibrated.pdb')

forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

system = forcefield.createSystem(
    topology=pdb.topology, 
    nonbondedMethod=PME,
    nonbondedCutoff=1*unit.nanometer,
    constraints=HBonds
    )

In [17]:
n_replicas = 4  # Number of temperature replicas.
T_min = 300.0 * unit.kelvin  # Minimum temperature.
T_max = 600.0 * unit.kelvin  # Maximum temperature.


# thermodynamic state holds information on ensemble parameters like temperture and pressure
reference_state = states.ThermodynamicState(system=system, temperature=T_min)


In [18]:
move = mcmc.GHMCMove(timestep=2.0*unit.femtoseconds, n_steps=50)

In [32]:
storage_path = 'replica_exchange.nc'
reporter = MultiStateReporter(storage_path, checkpoint_interval=10)

In [33]:
# Create sampler state with both positions and box vectors
sampler_state = states.SamplerState(
    positions=pdb.positions,
    box_vectors=system.getDefaultPeriodicBoxVectors()
)

simulation = ParallelTemperingSampler(
    mcmc_moves=move, 
    number_of_iterations=2
    )

simulation.create(
    thermodynamic_state=reference_state,
    sampler_states=[sampler_state],
    storage=reporter, 
    min_temperature=T_min,
    max_temperature=T_max, 
    n_temperatures=n_replicas)


simulation.run()


Exception ignored in: <function PhaseAnalyzer.__del__ at 0x7f56e0d03880>
Traceback (most recent call last):
  File "/home/jakub/anaconda3/envs/openmm/lib/python3.12/site-packages/openmmtools/multistate/multistateanalyzer.py", line 592, in __del__
    self.clear()
  File "/home/jakub/anaconda3/envs/openmm/lib/python3.12/site-packages/openmmtools/multistate/multistateanalyzer.py", line 1238, in clear
    super().clear()
  File "/home/jakub/anaconda3/envs/openmm/lib/python3.12/site-packages/openmmtools/multistate/multistateanalyzer.py", line 608, in clear
    self._invalidate_cache_values('reporter')
  File "/home/jakub/anaconda3/envs/openmm/lib/python3.12/site-packages/openmmtools/multistate/multistateanalyzer.py", line 791, in _invalidate_cache_values
    self._invalidate_cache_values(k)
  File "/home/jakub/anaconda3/envs/openmm/lib/python3.12/site-packages/openmmtools/multistate/multistateanalyzer.py", line 791, in _invalidate_cache_values
    self._invalidate_cache_values(k)
  File "/

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

In [34]:
simulation.read_status('replica_exchange.nc')

Status(iteration=2, target_error=None, is_completed=True)

In [1]:
from openmmtools.multistate import ParallelTemperingAnalyzer

analyzer = ParallelTemperingAnalyzer('replica_exchange.nc')


Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************

/home/jakub/anaconda3/envs/openmm/lib/python3.12/site-packages/openmmtools/multistate/multistateanalyzer.py:960: SyntaxWarning: invalid escape sequence '\l'
  """
/home/jakub/anaconda3/envs/o

ValueError: reporter must be a MultiStateReporter instance

In [21]:
import netCDF4
storage_path = 'replica_exchange.nc'
nc = netCDF4.Dataset(storage_path, 'r')
print(nc.variables.keys())

dict_keys(['last_iteration', 'analysis_particle_indices', 'options', 'metadata', 'states', 'energies', 'neighborhoods', 'accepted', 'proposed', 'timestamp'])


In [28]:
nc['states']

<class 'netCDF4._netCDF4.Variable'>
int32 states(iteration, replica)
    units: none
    long_name: states[iteration][replica] is the thermodynamic state index (0..n_states-1) of replica 'replica' of iteration 'iteration'.
unlimited dimensions: iteration
current shape = (51, 4)
filling on, default _FillValue of -2147483647 used

In [31]:
N_ITERATIONS = nc['states'].shape[0] - 1
N_REPLICAS = nc['states'].shape[1]

AttributeError: 'NoneType' object has no attribute 'variables'